In [1]:
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# Create a dataset from directories cars_fake and cars_real
from torchvision import datasets, transforms

# Create a dataset from directories cars_fake and cars_real
data_dirs = ['cars_fake', 'cars_real']

### Models

In [3]:
import torch.nn as nn 
import torch

from resnet50nodown import resnet50nodown


class DIMD(nn.Module):

    def __init__(self):
        super(DIMD, self).__init__()
        self.model = resnet50nodown(num_classes=1)

    def forward(self, x):
        return self.model(x)

    def load_weights(self, ckpt):
        state_dict = torch.load(ckpt, map_location='cpu')
        try:
            self.model.load_state_dict(state_dict['model'])
            if ('module._conv_stem.weight' in state_dict['model']) or ('module.fc.fc1.weight' in state_dict['model']) or ('module.fc.weight' in state_dict['model']):
                self.load_state_dict({key[7:]: state_dict['model'][key] for key in state_dict['model']})
            else:
                self.model.load_state_dict(state_dict['model'])

        except:
            print('Loading state dict failed. Trying to load without model prefix.')
            self.model.load_state_dict(state_dict)

    def predict(self, img):
        with torch.no_grad():
            logits = self.forward(img)
            return logits.sigmoid().flatten().tolist()

In [7]:
import clip 

import torch.nn as nn 
import torch


class UnivFD(nn.Module):
    def __init__(self, num_classes=1):
        super(UnivFD, self).__init__()
        self.model, _ = clip.load("ViT-L/14", device="cpu")
        self.fc = nn.Linear(768, num_classes)

    def forward(self, x):
        features = self.model.encode_image(x) 
        return self.fc(features)
    
    def load_weights(self, ckpt):
        state_dict = torch.load(ckpt, map_location='cpu')
        self.fc.load_state_dict(state_dict)

    def predict(self, img):
        with torch.no_grad():
            logits = self.forward(img)
            return logits.sigmoid().flatten().tolist()
        

In [8]:
DIMD_model = DIMD()
DIMD_model.load_weights('../DIMD_latent_weights/model_epoch_best.pth')

UnivFD_model = UnivFD()
UnivFD_model.load_weights('../UnivFD_weights/fc_weights.pth')

100%|███████████████████████████████████████| 890M/890M [03:42<00:00, 4.19MiB/s]


In [9]:
# Predict on a single image
from PIL import Image
from torchvision import transforms

def predict_image(image_path, model):
    img = Image.open(image_path)
    img = img.crop((0, 0, 224, 224))
    img = transforms.ToTensor()(img).unsqueeze(0)
    return model.predict(img)

# Predict on a single image
img_path = "cars_fake\\Acura Integra Type R 2001\\1.png"

result = predict_image(img_path, UnivFD_model)

print(result)

[0.9366340637207031]


In [11]:
import os as os
from sklearn.metrics import accuracy_score

#crawl through the directories cars_fake and cars_real and their subdirectories

def crawl_directories(directory):
    results = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith(".png") or file.endswith(".jpg"):
                results.append(os.path.join(root, file))
    return results

fake_images = crawl_directories("cars_fake")
real_images = crawl_directories("cars_real")

# Predict on all images
fake_results = [predict_image(img, UnivFD_model) for img in fake_images]
real_results = [predict_image(img, UnivFD_model) for img in real_images]

# Create a dataframe
df = pd.DataFrame(fake_results + real_results, columns=['score'])
df['label'] = ['fake'] * len(fake_results) + ['real'] * len(real_results)

df['predicted_label'] = df['score'].apply(lambda x: 'real' if x < 0.5 else 'fake')
accuracy = accuracy_score(df['label'], df['predicted_label'])
print(f"Model accuracy: {accuracy:.2f}")

Model accuracy: 0.64


In [12]:
df.head(100)

,score,label,predicted_label
0,0.415468,fake,real
1,0.936634,fake,fake
2,0.575678,fake,fake
3,0.045171,fake,real
4,0.312099,fake,real
...,...,...,...
95,0.005930,real,real
96,0.000831,real,real
97,0.007018,real,real
98,0.003862,real,real


In [ ]:
# Classwise accuracy

classwise_accuracy = df.groupby('label').apply(lambda x: accuracy_score(x['label'], x['predicted_label']))

C:\Users\asbjo\AppData\Local\Temp\ipykernel_23232\3985867426.py:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  classwise_accuracy = df.groupby('label').apply(lambda x: accuracy_score(x['label'], x['predicted_label']))


In [14]:
classwise_accuracy

label
fake    0.46875
real    0.81250
dtype: float64